# Pricing Models

In this notebook, I'll capture the different pricing models only.  I'll expect that other notebooks will use these models and thus this notebook will need to be run from other notebooks to be valuable

[TOC]


In [7]:
import pandas as pd
import numpy as np
import math
from scipy.stats import norm
from dateutil.parser import *
from datetime import *

# Common functions
def get_term(expiration, start=date.today()):
    
    if type(expiration) is str:
        expiration = parse(expiration).date()
    elif type(expiration) is pd.Timestamp:
        expiration = expiration.date()
        
    if type(start) is str:
        start = parse(start).date()
    
    return (expiration - start).days

## BlackSholes Model

The Black-Sholes pricing model can be calculated using the current price, strike price, volatility, risk free rate, and the expiration. Two approaches are outlined here - first given a specific date, we can parse and figure out the number of days. The second takes the number of days directly.

In [4]:
def BlackSholes(S0, strikePrice, volatility, rate, expiration = '12/31/2020'):
    term = get_term(expiration)
    return _BlackSholes(S0,strikePrice,volatility,rate, term, termUnits='days')

def _BlackSholes(S0, strikePrice, volatility, rate, term, termUnits='days'):
    # this is the log of the current price / strike price term
    logStockStrike = math.log(S0/strikePrice)
    # represents the sigma-squared divided by 2 term
    sigmaDiv2 = (volatility**2)/2
    # Convert days to part of years
    T = term/365 if termUnits == 'days' else term
    ert = math.exp(-rate*T)

    d1 = (logStockStrike + (rate+sigmaDiv2)*T) / (volatility*math.sqrt(T))
    d2 = d1 - volatility*math.sqrt(T)
    Nd1 = norm.cdf(d1)
    Nd2 = norm.cdf(d2)
    callPrice = S0*Nd1 - strikePrice*ert*Nd2

    Nnegd1 = norm.cdf(-d1)
    Nnegd2 = norm.cdf(-d2)
    putPrice = strikePrice*ert*Nnegd2 - S0*Nnegd1

    return (callPrice, putPrice)

In [5]:
# Notice when we call this function, that both the Call Price and the Put Price are returned
currentPrice = 333.45
volatility = 0.125
strikePrice = 340
riskFreeRate = 0.017
expiration = '12/31/2020'

callPrice, putPrice = BlackSholes(currentPrice, strikePrice, volatility, riskFreeRate, expiration)
print(f'Call Price: {callPrice:0.2f}')
print(f'Put Price: {putPrice:0.2f}')

Call Price: 12.61
Put Price: 15.19


In [8]:
callPrice, putPrice = BlackSholes(currentPrice, strikePrice, volatility, riskFreeRate, pd.Timestamp(year=2020,month=4, day=30))
print(f'Call Price: {callPrice:0.2f}')
print(f'Put Price: {putPrice:0.2f}')

Call Price: 0.40
Put Price: 6.84


## Cox-Ross-Rubenstein Binomial Model

Mathematical formula for estimating the value of an American option (exercisable at any time up to the expiration date). Used in determining if the early exercise of the option is advisable, it (unlike the Black-Scholes option pricing model) assumes that the price of the underlying asset follows binomial distribution. It divides the time to expiration into a certain number of intervals over which the price of the underlying contract is allowed to move up or down according to a specified probability. Invented in 1979 by the US mathematicians John Cox, Stephen Ross, and Mark Rubinstein.

Read more: http://www.businessdictionary.com/definition/Cox-Ross-Rubinstein-Option-Pricing-Model.html

In [35]:
def CRR_option_value(S0, strikePrice,volatility, rate, exp='12/31/2020'):
    ''' Cox-Ross-Rubinstein European option valuation
    
    S0: float, current price of the stock or index
    strikePrice: float, strike price
    volatility: float, volatility
    rate: float, risk-less short rate
    exp: string or datetime, the day of the expiration
    
    Note the call assumings the number of periods to oscillate is daily, therefore the number of days
    before the expiration is the number of intervals used in the calculations
    
    '''
    term = get_term(exp)
    return _CRR_option_value(S0, strikePrice, volatility, rate, num_intervals=term, term=term, termUnits='days')


def _CRR_option_value(S0, strikePrice,volatility, rate, num_intervals, term, termUnits='days'):
    ''' Cox-Ross-Rubinstein European option valuation
    
    S0: float, current price of the stock or index
    strikePrice: float, strike price
    volatility: float, volatility
    rate: float, risk-less short rate
    num_intervals: int, the number of points at which to reconsider the price of the security
    term: int, length of time to maturity in 'termUnits'
    termUnits: string, either 'days' or 'years'
    '''
    
    # Time components
    T = term/365 if termUnits == 'days' else termUnits # Convert days to part of years
    dt = T / num_intervals # Length of each interval
    df = math.exp(-rate*dt) # discount per interval
    
    # Binomial Parameters
    up_move = math.exp(volatility * math.sqrt(dt))
    down_move = 1 / up_move
    q = (math.exp(rate* dt)-down_move)/(up_move-down_move) # martingdale branch probability
    
    # Setting up the arrays
    # Since CRR closes in on the price either from underneath or by oscillation,
    #  these two arrays set up the greatest value (at a given time) or the smallest
    #  value at a given time in the future.
    mu = np.arange(num_intervals +1) # represents the up movements
    mu = np.resize(mu, (num_intervals+1, num_intervals+1))
    md = np.transpose(mu) # this is the down movements
    mu = up_move ** (mu-md) # this accounts for the up movements
    md = down_move ** md
    
    S= S0 * mu * md
    
    def innerValues(V):
        z = 0
        for t in range(num_intervals-1,-1,-1): # working backward
            V[0:num_intervals-z, t] = (q*V[0:num_intervals-z,t+1] + (1-q) *V[1:num_intervals-z+1,t+1]) * df
            z += 1
        return V[0,0]

    # Inner values - let's start with Call
    call_price = innerValues(np.maximum(S-strikePrice,0)) # if the value of the call is less than 0 then it is worthless
    # Inner values - now onto puts
    put_price = innerValues(np.maximum(strikePrice-S,0)) # if the value of the call is less than 0 then it is worthless
    
    
    return (call_price, put_price)
    

In [14]:
# Notice when we call this function, that both the Call Price and the Put Price are returned
currentPrice = 333.45
volatility = 0.125
strikePrice = 340
riskFreeRate = 0.017
expiration = '12/31/2020'

callPrice, putPrice = CRR_option_value(currentPrice, strikePrice, volatility, riskFreeRate,exp=expiration )
print(f'Call Price: {callPrice:0.2f}')
print(f'Put Price: {putPrice:0.2f}')


Call Price: 12.63
Put Price: 15.20
